In [1]:
import cv2
from IPython.display import clear_output
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from utils import ones_state, vis, state_ones

cam = cv2.VideoCapture(1)

In [2]:
from detector import Detector
d = Detector(64, 64, "weights-00-0.28.hdf5")

c:\Users\aabdo\miniconda3\envs\tic_tac_toe\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


model created with # params: 507858
loading model weights from path: weights-00-0.28.hdf5
loaded weights


In [3]:
# create tictac ai
import t3_ai
t3 = t3_ai.AI(weights_path="t3_weights-19-0.24.hdf5")

model created with # params: 353
loading model weights from path: t3_weights-19-0.24.hdf5
loaded weights


c:\Users\aabdo\miniconda3\envs\tic_tac_toe\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
def rc_from_int(a):
    r = int(np.floor(a/3.0))
    c = a%3
    return r,c

In [3]:
#!/usr/bin/env python3
from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe,
using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""

HUMAN = +1
COMP = -1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):
    """
    Function to heuristic evaluation of state.
    :param state: the state of the current board
    :return: +1 if the computer wins; -1 if the human wins; 0 draw
    """
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score


def wins(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    :param state: the state of the current board
    :param player: a human or a computer
    :return: True if the player wins
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    This function test if the human or computer wins
    :param state: the state of the current board
    :return: True if the human or computer wins
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    :param state: the state of the current board
    :return: a list of empty cells
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    A move is valid if the chosen cell is empty
    :param x: X coordinate
    :param y: Y coordinate
    :return: True if the board[x][y] is empty
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Set the move on board, if the coordinates are valid
    :param x: X coordinate
    :param y: Y coordinate
    :param player: the current player
    """
    if valid_move(x, y):
        board[x][y] = player

        return True
    else:
        return False


def minimax(state, depth, player):
    """
    AI function that choice the best move
    :param state: current state of the board
    :param depth: node index in the tree (0 <= depth <= 9),
    but never nine in this case (see iaturn() function)
    :param player: an human or a computer
    :return: a list with [the best row, best col, best score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    """
    Print the board on console
    :param state: current state of the board
    """

    chars = {
        HUMAN: h_choice,
        COMP: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)


def ai_turn(c_choice = 'O', h_choice = 'X'):
    """
    It calls the minimax function if the depth < 9,
    else it choices a random coordinate.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    if np.sum(np.array(board))==-1:
        return
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    # render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    # time.sleep(1)


def human_turn(c_choice, h_choice):
    """
    The Human plays choosing a valid move.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)
   
    # while move < 1 or move > 9:
    #     try:
    #         move = int(input('Use numpad (1..9): '))
    #         coord = moves[move]
    #         can_move = set_move(coord[0], coord[1], HUMAN)

    #         if not can_move:
    #             print('Bad move')
    #             move = -1
    #     except (EOFError, KeyboardInterrupt):
    #         print('Bye')
    #         exit()
    #     except (KeyError, ValueError):
    #         print('Bad choice')

threashold = 1000
counter = 0
def main_solve():
    """
    Main function that calls all functions
    """
    clean()
    h_choice = 'X'  # X or O
    c_choice = 'O'  # X or O
    first = 'N'  # if human is the first

    # Human chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting computer's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    render(board, c_choice, h_choice)

    if len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''
        
        human_turn(c_choice, h_choice)
        if np.sum(np.array(board))==0:
            counter += 1
            if counter > threashold:
                counter = 0
                ai_turn(c_choice, h_choice)
        else:
            counter = 0
        # ai_turn(c_choice, h_choice)

    # Game over message
    elif wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        # render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, COMP):
        clean()
        print(f'Computer turn [{c_choice}]')
        # render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        # render(board, c_choice, h_choice)
        print('DRAW!')

    # exit()


if __name__ == '__main__':
    # main()
    pass

In [4]:
import cv2
# from IPython.display import clear_output

# now let's initialize the list of reference point
ref_point = []
draw = False
def shape_selection(event, x, y, flags, param):
    # grab references to the global variables
    global ref_point, crop, draw
    # ix, iy = (0, 0)
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping is being performed
    if event == cv2.EVENT_LBUTTONDOWN:
        draw = True
        ref_point = [(x, y)]
        # image = clone.copy()
        # cv2.imshow("image", image)

    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        if draw:
            draw = False
            ref_point.append((x, y))
            ix, iy = (ref_point[0][0], ref_point[0][1])
            # calculate the side length of the square
            side_length = max(abs(x - ix), abs(y - iy))

            # determine the bottom-right corner of the square
            if x >= ix and y >= iy:
                ref_point[1] = (ix + side_length, iy + side_length)
            elif x >= ix and y < iy:
                ref_point[1] = (ix + side_length, iy - side_length)
            elif x < ix and y >= iy:
                ref_point[1] = (ix - side_length, iy + side_length)
            else:
                ref_point[1] = (ix - side_length, iy - side_length)

                        
            # cv2.imshow("image", image)




# load the image, clone it, and setup the mouse callback function
ret_val, image = cam.read()
# image = cv2.imread('download.png')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

croped_img = None
# keep looping until the 'q' key is pressed
rotation = 0
while True:
    # display the image and wait for a keypress
    ret_val, image = cam.read()
    for _ in range(rotation):
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    if len(ref_point)==2:
        ix, iy = (ref_point[0][0], ref_point[0][1])
        x, y = (ref_point[1][0], ref_point[1][1])
        # calculate the side length of the square
        side_length = max(abs(x - ix), abs(y - iy))
        if side_length >= 64:
            # draw a square around the region of interest
            thickness = 2
            cv2.rectangle(image, 
                    (ref_point[0][0] - thickness, ref_point[0][1] - thickness), 
                    (ref_point[1][0] + thickness, ref_point[1][1] + thickness), 
                    (0, 255, 0), thickness)
            croped_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            croped_img = croped_img[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
            croped_img = cv2.resize(croped_img, (64, 64))
            croped_img = cv2.adaptiveThreshold(croped_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11,2)
            croped_img = 255-croped_img
            croped_img = croped_img.astype(np.float64)
            croped_img /= 255

            clear_output(wait=True)
            cv2.imshow("cropped", cv2.resize(croped_img, (256, 256)))
            # plt.imshow(croped_img, cmap=plt.cm.gray)
            state = np.round(d.predict(croped_img))
            # print(ones_state(state))
            # vis(ones_state(state))
            state = ones_state(state)
            # state = [i*-1 for i in state]
            board = [state[i:i + 3] for i in range(0, len(state), 3)]
            main_solve()
            # print(board[2][0])
            
            # besta = t3.best_move(state)
            # r, c = rc_from_int(besta)
            # print(r,c)
    
    cv2.imshow("image", image)
    
    key = cv2.waitKey(1) & 0xFF

    # press 'r' to reset the window
    if key == ord("r"):
        # draw = True
        rotation += 1
        rotation %= 4
        # image = clone.copy()

    # if the 'c' key is pressed, break from the loop
    elif key == 27:
        break

# close all open windows
print(ref_point)
cv2.destroyAllWindows() 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step

---------------
| O || X || O |
---------------
| O || X || O |
---------------
| X || O || O |
---------------
Computer turn [O]
YOU LOSE!


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [1]:
#!/usr/bin/env python3
from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe,
using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""

HUMAN = -1
COMP = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):
    """
    Function to heuristic evaluation of state.
    :param state: the state of the current board
    :return: +1 if the computer wins; -1 if the human wins; 0 draw
    """
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score


def wins(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    :param state: the state of the current board
    :param player: a human or a computer
    :return: True if the player wins
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    This function test if the human or computer wins
    :param state: the state of the current board
    :return: True if the human or computer wins
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    :param state: the state of the current board
    :return: a list of empty cells
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    A move is valid if the chosen cell is empty
    :param x: X coordinate
    :param y: Y coordinate
    :return: True if the board[x][y] is empty
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Set the move on board, if the coordinates are valid
    :param x: X coordinate
    :param y: Y coordinate
    :param player: the current player
    """
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False


def minimax(state, depth, player):
    """
    AI function that choice the best move
    :param state: current state of the board
    :param depth: node index in the tree (0 <= depth <= 9),
    but never nine in this case (see iaturn() function)
    :param player: an human or a computer
    :return: a list with [the best row, best col, best score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    """
    Print the board on console
    :param state: current state of the board
    """

    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)


def ai_turn(c_choice, h_choice):
    """
    It calls the minimax function if the depth < 9,
    else it choices a random coordinate.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    # time.sleep(1)


def human_turn(c_choice, h_choice):
    """
    The Human plays choosing a valid move.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')


def main():
    """
    Main function that calls all functions
    """
    clean()
    h_choice = ''  # X or O
    c_choice = ''  # X or O
    first = ''  # if human is the first

    # Human chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting computer's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    while len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        human_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    # Game over message
    if wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, COMP):
        clean()
        print(f'Computer turn [{c_choice}]')
        render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        print('DRAW!')

    # exit()


if __name__ == '__main__':
    main()


Computer turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [O]

---------------
| X ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Computer turn [X]

---------------
| X || O ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [O]

---------------
| X || O ||   |
---------------
| X ||   ||   |
---------------
|   ||   ||   |
---------------
Computer turn [X]

---------------
| X || O ||   |
---------------
| X ||   ||   |
---------------
| O ||   ||   |
---------------
Human turn [O]

---------------
| X || O ||   |
---------------
| X || X ||   |
---------------
| O ||   ||   |
---------------
Computer turn [X]

---------------
| X || O ||   |
---------------
| X || X || O |
---------------
| O ||   ||   |
---------------
Computer turn [X]

---------------
| X || O ||   |
---------------
| X || X || O |
--------

: 